<a href="https://colab.research.google.com/github/RogsGROT/PF-ENA-/blob/main/AllRaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install andes==1.9.4

ERROR: Ignored the following yanked versions: 0.3.1, 0.5.7, 0.5.9, 0.5.11, 0.6.1, 0.6.2, 0.6.3, 0.6.4, 0.6.5, 0.6.6, 0.6.8, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.8.7, 0.8.8, 0.9.1, 0.9.2
ERROR: Could not find a version that satisfies the requirement andes==1.9.4 (from versions: 0.6.9, 0.9.3, 0.9.4, 0.9.5, 1.0.0, 1.0.1, 1.0.2, 1.0.3, 1.0.4, 1.0.5, 1.0.6, 1.0.7, 1.0.8, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.1.4, 1.1.5, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.2.4, 1.2.5, 1.2.6, 1.2.7, 1.2.8, 1.2.9, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.3.4, 1.3.5, 1.3.6, 1.3.8, 1.3.9, 1.3.10, 1.3.11, 1.3.12, 1.4.0, 1.4.1, 1.4.2, 1.4.3, 1.4.4, 1.5.0, 1.5.2, 1.5.3, 1.5.4, 1.5.5, 1.5.6, 1.5.8, 1.5.9, 1.5.10, 1.5.11, 1.5.12, 1.6.0, 1.6.1, 1.6.2, 1.6.3, 1.6.4, 1.6.5, 1.6.6, 1.7.0, 1.7.1, 1.7.2, 1.7.3, 1.7.4, 1.7.5, 1.7.6, 1.7.7, 1.7.8, 1.8.0, 1.8.1, 1.8.2, 1.8.3, 1.8.4, 1.8.5, 1.8.6, 1.8.7, 1.8.8, 1.8.9, 1.8.10, 1.9.0, 1.9.1, 1.9.2, 1.9.3)
ERROR: No matching distribution found for andes==1.9.4


In [50]:
import andes
import numpy as np
import pandas as pd
import logging
import os

# Configure ANDES logging
try:
    andes.config_logger(stream_level=logging.WARNING)
except AttributeError:
    try:
        andes.main.config_logger(stream_level=logging.WARNING)
    except Exception as e_log:
        print(f"Could not configure ANDES logger: {e_log}. Proceeding with default logging.")

def build_ena_matrix_from_power_system(raw_file_path):
    print(f"Loading PSS/E file and running power flow: {raw_file_path}")
    system = None
    try:
        print("Attempting andes.run()...")
        # Ensure output files like .lst are generated for future reference if needed
        system = andes.run(raw_file_path, default_config=True, no_output=False)
        print(f"andes.run() completed. Returned object type: {type(system)}")
        output_dir = system.config.output_path if hasattr(system, 'config') and hasattr(system.config, 'output_path') else os.getcwd()
        print(f"Output files (e.g., _out.txt, _out.lst) should be in: {output_dir}")
    except Exception as e_run:
        print(f"Exception directly from andes.run() call: {type(e_run).__name__}: {e_run}")
        import traceback; traceback.print_exc()
        return None

    sbase = 100.0
    sbase_found_explicitly = False

    try:
        if system is None: print("Error: andes.run() returned None."); return None
        if hasattr(system, 'exit_code') and system.exit_code != 0:
            print(f"ANDES run error (exit_code {system.exit_code})."); return None
        print(f"ANDES run successful (exit_code {getattr(system, 'exit_code', 'N/A')}).")

        if hasattr(system, 'config') and hasattr(system.config, 'mva') and system.config.mva is not None:
            sbase = system.config.mva
            sbase_found_explicitly = True
            print(f"Using SBASE = {sbase} MVA (from system.config.mva).")
        else:
            print(f"CRITICAL WARNING: SBASE not found in system.config.mva. Using default {sbase} MVA.")

        if hasattr(system, 'Bus') and hasattr(system.Bus, 'v') and \
           hasattr(system.Bus.v, 'v') and system.Bus.v.v is not None and \
           isinstance(system.Bus.v.v, np.ndarray) and system.Bus.v.v.size > 0 :
            print(f"Bus voltage magnitudes (system.Bus.v.v) are present. Shape: {system.Bus.v.v.shape}")
        else:
            print("Warning: Bus voltage magnitudes (system.Bus.v.v) not found or empty.")

    except Exception as e_check_basic:
        print(f"Error during basic checks of 'system' object: {type(e_check_basic).__name__}: {e_check_basic}")
        import traceback; traceback.print_exc()
        return None

    print("Extracting data for ENA matrix...")

    es_node = 'ES (ExternalSource)'
    load_node = 'LOAD (SystemLoad)'
    loss_node = 'LOSS (SystemLosses)'
    gen_nodes_dict = {}
    gen_data_for_matrix = {}

    bus_idx_to_name_map = {}
    bus_names_list_for_nodes = []
    if hasattr(system, 'Bus') and hasattr(system.Bus, 'idx') and hasattr(system.Bus.idx, 'v') and \
       hasattr(system.Bus, 'name') and hasattr(system.Bus.name, 'v'):
        try:
            bus_indices = system.Bus.idx.v
            bus_names_raw = system.Bus.name.v
            if len(bus_indices) == len(bus_names_raw):
                for i in range(len(bus_indices)):
                    bus_idx_int = int(bus_indices[i])
                    bus_name_str = str(bus_names_raw[i]).strip()
                    bus_idx_to_name_map[bus_idx_int] = bus_name_str
                    if bus_name_str not in bus_names_list_for_nodes:
                        bus_names_list_for_nodes.append(bus_name_str)
                bus_names_list_for_nodes.sort()
                print(f"  Created bus idx to name map for {len(bus_idx_to_name_map)} buses.")
            else: print("  Warning: Mismatch len(Bus.idx.v) and len(Bus.name.v).")
        except Exception as e_bus_map: print(f"  Warning: Error creating bus_idx_to_name_map: {e_bus_map}")
    else: print("  CRITICAL Warning: system.Bus.idx.v or system.Bus.name.v not available. Bus mapping will fail.")

    final_bus_nodes_for_matrix = bus_names_list_for_nodes

    print(f"Attempting to collect generators...")
    potential_gen_types = ['PV', 'SWING', 'GENCLS', 'GENROU', 'SYN']
    for model_name in potential_gen_types:
        if hasattr(system, model_name):
            src_group = getattr(system, model_name)
            if not (hasattr(src_group, 'n') and src_group.n > 0 and \
                    hasattr(src_group, 'name') and hasattr(src_group.name, 'v') and \
                    hasattr(src_group, 'bus') and hasattr(src_group.bus, 'v')):
                continue

            pg_val_array = None; power_attr_name_used = "None"
            # Try to get Algeb variable 'p' or 'PG' via DAE array (most robust for results)
            # These Algeb objects (src_group.p or src_group.PG) should have an '.a' attribute
            if hasattr(src_group, 'p') and hasattr(src_group.p, 'a') and hasattr(system, 'dae') and hasattr(system.dae, 'y'):
                try: pg_val_array = system.dae.y[src_group.p.a]; power_attr_name_used = 'p (from dae.y)'
                except Exception as e: print(f"Could not get system.dae.y[system.{model_name}.p.a]: {e}")
            elif hasattr(src_group, 'PG') and hasattr(src_group.PG, 'a') and hasattr(system, 'dae') and hasattr(system.dae, 'y'):
                try: pg_val_array = system.dae.y[src_group.PG.a]; power_attr_name_used = 'PG (from dae.y)'
                except Exception as e: print(f"Could not get system.dae.y[system.{model_name}.PG.a]: {e}")

            # Fallback to .v if .a access failed or not applicable (e.g. for ConstService like PV.p)
            if pg_val_array is None:
                if model_name == 'PV' and hasattr(src_group, 'p') and hasattr(src_group.p, 'v'):
                    pg_val_array = src_group.p.v; power_attr_name_used = 'p.v (ConstService)'
                elif hasattr(src_group, 'PG') and hasattr(src_group.PG, 'v'): # General GENs often have PG as Algeb, .v should work too
                    pg_val_array = src_group.PG.v; power_attr_name_used = 'PG.v'


            if pg_val_array is None:
                if src_group.n > 0:
                    print(f"  Skipping system.{model_name}: No suitable active power attribute values found (tried via dae.y and direct .v).")
                continue

            print(f"  Processing system.{model_name} (found {src_group.n}), using attribute values from '{power_attr_name_used}' for power.")
            for i in range(src_group.n):
                is_online = True
                if hasattr(src_group, 'u') and hasattr(src_group.u, 'v') and i < len(src_group.u.v):
                    if src_group.u.v[i] == 0: is_online = False
                if not is_online: continue

                gen_id_raw = src_group.name.v[i]
                gen_name_base = str(gen_id_raw if isinstance(gen_id_raw, str) and gen_id_raw.strip() else f"{model_name}_ID{gen_id_raw}")

                pg_val_mw = pg_val_array[i] * sbase
                bus_andes_idx = int(src_group.bus.v[i])
                bus_name_for_gen = bus_idx_to_name_map.get(bus_andes_idx, f"UNKNOWN_BUS_IDX_{bus_andes_idx}")

                if "UNKNOWN_BUS" in bus_name_for_gen:
                     print(f"    Warning: Gen '{gen_name_base}' (Bus Idx {bus_andes_idx}) - bus index not in map. Using placeholder.")

                gen_name_for_matrix = gen_name_base
                count = 0
                while gen_name_for_matrix in gen_nodes_dict:
                    count += 1; gen_name_for_matrix = f"{gen_name_base}_{count}"

                if abs(pg_val_mw) > 1e-6:
                    gen_nodes_dict[gen_name_for_matrix] = None
                    gen_data_for_matrix[gen_name_for_matrix] = {'pg': pg_val_mw, 'bus_name': bus_name_for_gen}

    final_gen_nodes_for_matrix = sorted(list(gen_nodes_dict.keys()))

    ena_node_names = [es_node] + final_gen_nodes_for_matrix + final_bus_nodes_for_matrix + [load_node, loss_node]
    num_nodes = len(ena_node_names)
    ena_matrix = np.zeros((num_nodes, num_nodes))
    node_to_idx = {name: i for i, name in enumerate(ena_node_names)}

    print(f"Identified {len(final_gen_nodes_for_matrix)} active generators: {final_gen_nodes_for_matrix}")
    print(f"Identified {len(final_bus_nodes_for_matrix)} buses: {final_bus_nodes_for_matrix[:5]}...")
    if len(final_gen_nodes_for_matrix) == 0: print("CRITICAL WARNING: No active generators extracted for ENA matrix.")
    print(f"Total ENA nodes: {num_nodes}")

    idx_es = node_to_idx[es_node]
    for gen_name, data in gen_data_for_matrix.items():
        idx_gen = node_to_idx.get(gen_name)
        idx_bus = node_to_idx.get(data['bus_name'])
        if idx_gen is not None:
            ena_matrix[idx_es, idx_gen] = data['pg']
            if idx_bus is not None:
                ena_matrix[idx_gen, idx_bus] = data['pg']
            elif "UNKNOWN_BUS" not in data['bus_name']:
                print(f"  ERROR ENA POPULATION: Mapped bus name '{data['bus_name']}' for generator '{gen_name}' not found in final ENA node list.")
        else:
            print(f"  ERROR ENA POPULATION: Generator name '{gen_name}' not found in ENA node list.")

    idx_loss_node = node_to_idx[loss_node]

    # --- Branch Processing (Line & Transformer Flows & Losses) using DAE array access ---
    print(f"\nAttempting to process branches (Lines and Transformers) using system.dae.y access...")
    processed_branches = 0
    for branch_group_name_iter in ['Line', 'Transformer']:
        if hasattr(system, branch_group_name_iter):
            branch_group = getattr(system, branch_group_name_iter)

            if not (hasattr(branch_group, 'n') and branch_group.n > 0 and \
                    hasattr(branch_group, 'name') and hasattr(branch_group.name, 'v') and \
                    hasattr(branch_group, 'bus1') and hasattr(branch_group.bus1, 'v') and \
                    hasattr(branch_group, 'bus2') and hasattr(branch_group.bus2, 'v')):
                if hasattr(branch_group, 'n') and branch_group.n > 0:
                    print(f"  Skipping system.{branch_group_name_iter} group: missing basic name/bus attributes.")
                continue

            # Check for P1 and P2 Algeb objects and their .a addresses
            if not (hasattr(branch_group, 'P1') and hasattr(branch_group.P1, 'a') and \
                    hasattr(branch_group, 'P2') and hasattr(branch_group.P2, 'a') and \
                    hasattr(system, 'dae') and hasattr(system.dae, 'y')):
                if branch_group.n > 0:
                    print(f"  CRITICAL SKIP for system.{branch_group_name_iter}: Cannot access P1.a/P2.a or system.dae.y for flows.")
                continue

            print(f"  Processing system.{branch_group_name_iter} with {branch_group.n} elements using DAE arrays.")
            try:
                p1_values_pu = system.dae.y[branch_group.P1.a]
                p2_values_pu = system.dae.y[branch_group.P2.a]
            except Exception as e_dae_access:
                print(f"    Error accessing flow values from system.dae.y for {branch_group_name_iter}: {e_dae_access}")
                continue # Skip this whole branch group if DAE access fails

            for i in range(branch_group.n):
                is_online = True
                if hasattr(branch_group, 'u') and hasattr(branch_group.u, 'v') and i < len(branch_group.u.v):
                    if branch_group.u.v[i] == 0: is_online = False
                if not is_online: continue

                branch_name_raw = branch_group.name.v[i]
                branch_name = str(branch_name_raw if isinstance(branch_name_raw, str) and branch_name_raw.strip() else f"{branch_group_name_iter}_ID{branch_name_raw}")
                bus1_andes_idx = int(branch_group.bus1.v[i])
                bus2_andes_idx = int(branch_group.bus2.v[i])

                bus1_name = bus_idx_to_name_map.get(bus1_andes_idx)
                bus2_name = bus_idx_to_name_map.get(bus2_andes_idx)

                if not (bus1_name and bus2_name):
                    print(f"    Warning: Could not map bus names for {branch_name} (Buses {bus1_andes_idx}, {bus2_andes_idx}). Skipping.")
                    continue

                idx_bus1_ena = node_to_idx.get(bus1_name)
                idx_bus2_ena = node_to_idx.get(bus2_name)
                if idx_bus1_ena is None or idx_bus2_ena is None: continue

                p_from_bus1_mw = p1_values_pu[i] * sbase
                p_from_bus2_mw = p2_values_pu[i] * sbase
                branch_loss_mw = p_from_bus1_mw + p_from_bus2_mw

                if p_from_bus1_mw > 1e-6:
                    ena_matrix[idx_bus1_ena, idx_bus2_ena] += p_from_bus1_mw
                # Check if flow is from bus2 to bus1. P2 is power from bus2 *into* line.
                # If P1 is negative (bus1 receives) and P2 is positive (bus2 sends), then flow is from bus2 to bus1.
                elif p_from_bus2_mw > 1e-6:
                    ena_matrix[idx_bus2_ena, idx_bus1_ena] += p_from_bus2_mw

                if abs(branch_loss_mw) > 1e-6:
                     ena_matrix[idx_bus1_ena, idx_loss_node] += branch_loss_mw / 2.0
                     ena_matrix[idx_bus2_ena, idx_loss_node] += branch_loss_mw / 2.0
                processed_branches +=1

    if processed_branches == 0:
        print("CRITICAL WARNING: No branch flows processed. Inter-bus flows and losses will be missing.")

    # --- Load Processing ---
    idx_load_node = node_to_idx[load_node]
    print(f"\nAttempting to process loads from StaticLoad models...")
    processed_loads_count = 0

    if hasattr(system, 'StaticLoad') and hasattr(system.StaticLoad, 'models') and \
       isinstance(system.StaticLoad.models, dict) and system.StaticLoad.models:
        print(f"  Processing loads from system.StaticLoad.models ({len(system.StaticLoad.models)} model instances)...")

        for load_instance_key, load_model in system.StaticLoad.models.items():
            load_instance_name = str(load_instance_key)

            if not (hasattr(load_model, 'bus') and hasattr(load_model.bus, 'v')):
                print(f"    Skipping StaticLoad instance '{load_instance_name}': missing 'bus.v'.")
                continue

            active_power_pu_instance = 0.0
            power_attr_found = False

            # Prefer calculated Algeb 'P' if available
            if hasattr(load_model, 'P') and hasattr(load_model.P, 'a') and \
               hasattr(system, 'dae') and hasattr(system.dae, 'y'):
                try:
                    # An Algeb on an instance will have a scalar address into its own component's part of dae.y
                    # The .a for an instance's variable is typically an integer, not an array.
                    # And the value might already be in load_model.P.v directly after PF.
                    if hasattr(load_model.P, 'v') and load_model.P.v is not None:
                         # Value of an Algeb on instance is often just .v, not .v[0] unless it's an array Algeb (rare for instance P)
                        active_power_pu_instance = load_model.P.v
                        power_attr_found = True
                        print(f"    Processing StaticLoad '{load_instance_name}' using calculated P.v = {active_power_pu_instance:.4f} PU")
                    elif load_model.P.a is not None : # If .v not directly populated, try DAE array with address
                        # Ensure the address is an integer and valid for system.dae.y
                        addr = load_model.P.a
                        if isinstance(addr, (np.ndarray, list)) and len(addr) > 0: addr = addr[0] # Take first if array
                        if isinstance(addr, (int, np.integer)) and addr < len(system.dae.y):
                            active_power_pu_instance = system.dae.y[addr]
                            power_attr_found = True
                            print(f"    Processing StaticLoad '{load_instance_name}' using calculated P (from dae.y[{addr}]) = {active_power_pu_instance:.4f} PU")
                except Exception as e_load_P:
                    print(f"      Error accessing P.v or dae.y[P.a] for load '{load_instance_name}': {e_load_P}")

            # Fallback to parameter 'p0' (lowercase from your introspection of 'PQ' instance)
            if not power_attr_found and hasattr(load_model, 'p0') and hasattr(load_model.p0, 'v'):
                p0_val_raw = load_model.p0.v
                # p0.v for a NumParam on an instance is typically a scalar or 1-element array.
                active_power_pu_instance = p0_val_raw[0] if isinstance(p_val_pu_instance_raw, (np.ndarray, list)) and len(p_val_pu_instance_raw)>0 else p_val_pu_instance_raw

                if isinstance(active_power_pu_instance, (int, float, np.number)):
                    # Apply status if using parameter
                    status_val = 1.0
                    if hasattr(load_model, 'u') and hasattr(load_model.u, 'v'):
                        u_val_raw = load_model.u.v
                        status_val = u_val_raw[0] if isinstance(u_val_raw, (np.ndarray, list)) and len(u_val_raw)>0 else u_val_raw
                    active_power_pu_instance *= status_val
                    power_attr_found = True
                    print(f"    Processing StaticLoad '{load_instance_name}' using parameter p0.v*status = {active_power_pu_instance:.4f} PU")
                else:
                    active_power_pu_instance = 0.0 # Reset if not a number

            if not power_attr_found:
                print(f"    Skipping StaticLoad instance '{load_instance_name}': No P.v (Algeb result) or p0.v (NumParam) found.")
                continue

            # Get bus connection
            bus_val_raw = load_model.bus.v
            load_bus_andes_idx = int(bus_val_raw[0] if isinstance(bus_val_raw, (np.ndarray, list)) and len(bus_val_raw)>0 else bus_val_raw)
            load_bus_name = bus_idx_to_name_map.get(load_bus_andes_idx)

            if load_bus_name is None:
                print(f"    Warning: Could not map bus for StaticLoad instance '{load_instance_name}' at bus_idx {load_bus_andes_idx}.")
                continue

            idx_bus_ena = node_to_idx.get(load_bus_name)
            if idx_bus_ena is not None:
                load_val_mw = active_power_pu_instance * sbase
                if load_val_mw > 1e-6:
                    ena_matrix[idx_bus_ena, idx_load_node] += load_val_mw
                    processed_loads_count += 1
            else:
                 print(f"    Warning: ENA index for bus '{load_bus_name}' (for load '{load_instance_name}') not found.")

        if processed_loads_count > 0:
            print(f"  Processed {processed_loads_count} load entries from StaticLoad models.")

    if processed_loads_count == 0:
        print("  CRITICAL WARNING: No loads successfully processed from any source.")

    print("\nENA matrix construction complete.")
    return pd.DataFrame(ena_matrix, index=ena_node_names, columns=ena_node_names)

# --- Main Execution ---
if __name__ == '__main__':
    try: andes.config_logger(stream_level=logging.WARNING)
    except: pass

    raw_file_path = input("Enter the path to your PSS/E .raw file: ")
    ena_df = build_ena_matrix_from_power_system(raw_file_path)

    if ena_df is not None:
        print("\nENA Flow Matrix (values typically in MW):")
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        pd.set_option('display.precision', 3)
        print(ena_df)
        try:
            ena_df.to_csv('ena_matrix_power_system.csv')
            print("\nENA Matrix saved to ena_matrix_power_system.csv")
        except Exception as e: print(f"\nError saving ENA matrix to CSV: {e}")
    else:
        print("Failed to generate ENA matrix.")

Enter the path to your PSS/E .raw file: /content/ieee14.raw


INFO:andes:Working directory: "/content"
DEBUG:andes:Found files: ['/content/ieee14.raw']


Loading PSS/E file and running power flow: /content/ieee14.raw
Attempting andes.run()...


INFO:andes.system:> Reloaded generated Python code of module "pycode".
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
DEBUG:andes.io.psse:PSSE raw version 32 detected
DEBUG:andes.io:Input format guessed as psse.
INFO:andes.io:Parsing input file "/content/ieee14.raw"...
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
INFO:andes.io.psse:  IEEE 14 BUS TEST CASE
INFO:andes.io.psse:  03/06/14 CONTO           100.0  1962 W
INFO:andes.io:Input file parsed in 0.0358 seconds.
DEBUG:andes.system:Setting internal address for Bus
DEBUG:andes.system:Setting internal address for PQ
DEBUG:andes.system:Setting internal address for PV
DEBUG:andes.system:Setting internal address for Slack
DEBUG:andes.system:Setting internal address for Shunt
DEBUG:andes.system:Setting internal address for Line
DEBUG:andes.system:Setting internal address for Area
DEBUG:andes.core.connman:No need to update connectivity

-> Single process finished in 0.4583 seconds.
andes.run() completed. Returned object type: <class 'andes.system.System'>
Output files (e.g., _out.txt, _out.lst) should be in: /content
ANDES run successful (exit_code 0).
Using SBASE = 100.0 MVA (from system.config.mva).
Bus voltage magnitudes (system.Bus.v.v) are present. Shape: (14,)
Extracting data for ENA matrix...
  Created bus idx to name map for 14 buses.
Attempting to collect generators...
  Processing system.PV (found 4), using attribute values from 'p.v (ConstService)' for power.
Identified 4 active generators: ['PV_ID2', 'PV_ID3', 'PV_ID4', 'PV_ID5']
Identified 14 buses: ['BUS1', 'BUS10', 'BUS11', 'BUS12', 'BUS13']...
Total ENA nodes: 21

Attempting to process branches (Lines and Transformers) using system.dae.y access...
  CRITICAL SKIP for system.Line: Cannot access P1.a/P2.a or system.dae.y for flows.
CRITICAL WARNING: No branch flows processed. Inter-bus flows and losses will be missing.

Attempting to process loads from S

NameError: name 'p_val_pu_instance_raw' is not defined

In [48]:
import andes
import pandas as pd
import numpy as np

# It's good practice to import specific classes if known paths are confirmed
# For GroupBase, based on typical ANDES structure and if models/group.py was provided:
try:
    from andes.models.group import GroupBase
except ImportError:
    # Fallback if the exact path is different or not available,
    # the isinstance check might not be as specific later.
    print("Warning: andes.models.group.GroupBase not found, isinstance check for groups might be affected.")
    GroupBase = None # Define it as None if not importable

def run_power_flow_and_print_results(case_file_path):
    """
    Loads a power system case, runs power flow, and prints results
    for generator powers, load powers, and line flows by accessing
    the solved algebraic variables, parameters, and service values.
    This script is designed to be general for ANDES models.
    """
    ss = None
    try:
        print(f"Loading case file: {case_file_path}")
        ss = andes.load(case_file_path, setup=True, no_output=True)
        print("Case file loaded successfully.")

        print("\nRunning Power Flow...")
        pf_success = ss.PFlow.run()

        if not pf_success:
            print(f"Power flow calculation did not converge or failed. Exit code: {getattr(ss.PFlow, 'exit_code', 'N/A')}")
            return
        print("Power flow calculation completed successfully.")

        def get_values_as_series(data_provider_object, value_attribute_name, index_provider_model):
            if data_provider_object is None:
                return pd.Series(dtype=float)

            if not (index_provider_model and hasattr(index_provider_model, 'idx') and hasattr(index_provider_model.idx, 'v')):
                return pd.Series(dtype=float)

            if hasattr(data_provider_object, value_attribute_name):
                value_array = getattr(data_provider_object, value_attribute_name)
                idx_array_raw = index_provider_model.idx.v
                idx_array = [str(item) for item in idx_array_raw]

                if isinstance(value_array, (np.ndarray, list)) and len(value_array) == len(idx_array):
                    try:
                        if pd.Series(idx_array).is_unique:
                            return pd.Series(data=value_array, index=idx_array)
                        else:
                            return pd.Series(data=value_array, index=list(idx_array))
                    except Exception as e_series:
                        obj_name_str = getattr(data_provider_object, 'name', str(type(data_provider_object)))
                        model_name_str = getattr(index_provider_model, 'class_name', 'UnknownModel')
                        print(f"  Warning: Could not create series for {model_name_str}'s {obj_name_str}.{value_attribute_name}: {e_series}")

            return pd.Series(dtype=float)

        print("\n--- Generator Powers ---")

        # PV Generators
        if hasattr(ss, 'PV') and ss.PV.n > 0:
            print("\nPV Generators:")
            pv_df_full = ss.PV.as_df()
            p_series = get_values_as_series(ss.PV.p0, 'v', ss.PV)
            q_series = get_values_as_series(ss.PV.q, 'v', ss.PV)

            for _, row in pv_df_full.iterrows():
                dev_id_str = str(row['idx'])
                p_val = p_series.get(dev_id_str, float('nan'))
                q_val = q_series.get(dev_id_str, float('nan'))
                print(f"  Generator: {row.get('name', dev_id_str)} (ID: {dev_id_str}), Bus: {row.get('bus', 'N/A')}, "
                      f"P: {p_val:.4f} MW (setpoint), Q: {q_val:.4f} MVAr (solved)")
        else:
            print("\n  No PV generators found in the case.")

        # Slack Generators
        if hasattr(ss, 'Slack') and ss.Slack.n > 0:
            print("\nSlack Generators:")
            slack_df_full = ss.Slack.as_df()
            p_series = get_values_as_series(ss.Slack.p, 'v', ss.Slack)
            q_series = get_values_as_series(ss.Slack.q, 'v', ss.Slack)

            for _, row in slack_df_full.iterrows():
                dev_id_str = str(row['idx'])
                p_val = p_series.get(dev_id_str, float('nan'))
                q_val = q_series.get(dev_id_str, float('nan'))
                print(f"  Generator: {row.get('name', dev_id_str)} (ID: {dev_id_str}), Bus: {row.get('bus', 'N/A')}, "
                      f"P: {p_val:.4f} MW (solved), Q: {q_val:.4f} MVAr (solved)")
        else:
            print("\n  No Slack generators found in the case.")

        # Synchronous Generators (GENROU, GENCLS from GENBase)
        for gen_model_name in ['GENROU', 'GENCLS']:
            if hasattr(ss, gen_model_name) and getattr(ss, gen_model_name).n > 0:
                print(f"\n{gen_model_name} Generators:")
                model_obj = getattr(ss, gen_model_name)
                df_full = model_obj.as_df()
                p_series = get_values_as_series(model_obj.Pe, 'v', model_obj)
                q_series = get_values_as_series(model_obj.Qe, 'v', model_obj)

                for _, row in df_full.iterrows():
                    dev_id_str = str(row['idx'])
                    p_val = p_series.get(dev_id_str, float('nan'))
                    q_val = q_series.get(dev_id_str, float('nan'))
                    print(f"  Generator: {row.get('name', dev_id_str)} (ID: {dev_id_str}), Bus: {row.get('bus', 'N/A')}, "
                          f"P: {p_val:.4f} MW, Q: {q_val:.4f} MVAr")
            else:
                print(f"\n  No {gen_model_name} generators found in the case.")

        # StaticGen (distinct type)
        if hasattr(ss, 'StaticGen') and ss.StaticGen.n > 0:
            sg_df_full = ss.StaticGen.as_df()

            ids_pv_str = [str(item) for item in (ss.PV.idx.v if hasattr(ss, 'PV') and hasattr(ss.PV, 'idx') else [])]
            ids_slack_str = [str(item) for item in (ss.Slack.idx.v if hasattr(ss, 'Slack') and hasattr(ss.Slack, 'idx') else [])]
            already_handled_ids = set(ids_pv_str + ids_slack_str)

            actual_staticgen_rows = []
            if 'idx' in sg_df_full.columns:
                for _, row in sg_df_full.iterrows():
                    if str(row['idx']) not in already_handled_ids:
                        actual_staticgen_rows.append(row)

            if actual_staticgen_rows:
                print("\nStaticGen Generators (distinct type):")
                # For distinct StaticGen devices, directly use p0 and q0 parameters from their DataFrame.
                # This assumes pure StaticGen devices define their output via p0, q0 if algebraic 'p'/'q' are not directly on ss.StaticGen
                for row_data in actual_staticgen_rows:
                    dev_id_str = str(row_data['idx'])
                    p_val = row_data.get('p0', float('nan'))
                    q_val = row_data.get('q0', float('nan'))
                    print(f"  Generator: {row_data.get('name', dev_id_str)} (ID: {dev_id_str}), Bus: {row_data.get('bus', 'N/A')}, "
                          f"P: {p_val:.4f} MW, Q: {q_val:.4f} MVAr")
            elif not ( (hasattr(ss, 'PV') and ss.PV.n > 0 and ss.PV.group == 'StaticGen') or \
                       (hasattr(ss, 'Slack') and ss.Slack.n > 0 and ss.Slack.group == 'StaticGen') ):
                 # Only print "No distinct" if StaticGen group was empty or didn't just group PV/Slack
                 print("\n  No distinct StaticGen model instances found.")


        print("\n--- Load Powers (PQ model) ---")
        if hasattr(ss, 'PQ') and ss.PQ.n > 0:
            pq_df_full = ss.PQ.as_df()
            p_series = get_values_as_series(ss.PQ.Ppf, 'v', ss.PQ)
            q_series = get_values_as_series(ss.PQ.Qpf, 'v', ss.PQ)

            for _, row in pq_df_full.iterrows():
                dev_id_str = str(row['idx'])
                p_val = p_series.get(dev_id_str, float('nan'))
                q_val = q_series.get(dev_id_str, float('nan'))
                print(f"  Load: {row.get('name', dev_id_str)} (ID: {dev_id_str}), Bus: {row.get('bus', 'N/A')}, "
                      f"P: {p_val:.4f} MW, Q: {q_val:.4f} MVAr")
        else:
            print("  No PQ loads found in the case.")

        line_model_name = 'Line'
        if not hasattr(ss, line_model_name) or getattr(ss, line_model_name).n == 0:
            line_model_name = 'ACLine'

        print(f"\n--- Line Power Flows ({line_model_name} model) ---")
        if hasattr(ss, line_model_name) and getattr(ss, line_model_name).n > 0:
            line_model_obj = getattr(ss, line_model_name)
            line_df_full = line_model_obj.as_df()

            # Corrected calls for line flows using positional arguments for the helper
            p_at_bus1_series = get_values_as_series(line_model_obj.a1, 'e', line_model_obj)
            q_at_bus1_series = get_values_as_series(line_model_obj.v1, 'e', line_model_obj)
            p_at_bus2_series = get_values_as_series(line_model_obj.a2, 'e', line_model_obj)
            q_at_bus2_series = get_values_as_series(line_model_obj.v2, 'e', line_model_obj)

            for _, row in line_df_full.iterrows():
                dev_id_str = str(row['idx'])
                p_inj_bus1 = p_at_bus1_series.get(dev_id_str, float('nan'))
                q_inj_bus1 = q_at_bus1_series.get(dev_id_str, float('nan'))
                p_inj_bus2 = p_at_bus2_series.get(dev_id_str, float('nan'))
                q_inj_bus2 = q_at_bus2_series.get(dev_id_str, float('nan'))

                print(f"  Line: {row.get('name', dev_id_str)} (ID: {dev_id_str}) (Bus1: {row.get('bus1','N/A')} to Bus2: {row.get('bus2','N/A')})")
                print(f"    Injection at Bus {row.get('bus1','N/A')} from Line: P = {p_inj_bus1:.4f} MW, Q = {q_inj_bus1:.4f} MVAr")
                print(f"    Injection at Bus {row.get('bus2','N/A')} from Line: P = {p_inj_bus2:.4f} MW, Q = {q_inj_bus2:.4f} MVAr")
        else:
            print(f"  No {line_model_name} model found or no instances in the case.")

    except FileNotFoundError:
        print(f"Error: Case file not found at '{case_file_path}'. Please provide a valid path.")
    except ImportError as ie:
        print(f"Error: ANDES library or a dependency not found: {ie}.")
    except AttributeError as ae:
        print(f"An AttributeError occurred: {ae}.")
        if ss and hasattr(ss, 'PFlow') and hasattr(ss.PFlow, 'exit_code') and ss.PFlow.exit_code != 0 :
             print(f"Power flow convergence issue might be the cause. Exit code: {ss.PFlow.exit_code}")
        elif ss:
            print("Debug: Available components in the system object (ss):")
            for comp_name in dir(ss):
                if not comp_name.startswith('_'):
                    comp_obj = getattr(ss, comp_name, None)
                    # Corrected isinstance check for GroupBase
                    if GroupBase and isinstance(comp_obj, GroupBase) and hasattr(comp_obj, 'n') and comp_obj.n > 0:
                         print(f"  - {comp_name} (Instances: {comp_obj.n}, Type: {type(comp_obj)})")
                         for var_container_name in ['algebs', 'params', 'services', 'states', 'vars_ext', 'vars_int']:
                             if hasattr(comp_obj, var_container_name):
                                 container = getattr(comp_obj, var_container_name)
                                 if container and isinstance(container, dict) and container:
                                     print(f"    {var_container_name}: {list(container.keys())}")
                         if comp_name == line_model_name: # Use dynamic line_model_name
                             for ext_algeb_name in ['a1', 'v1', 'a2', 'v2']:
                                 if hasattr(comp_obj, ext_algeb_name):
                                     ext_algeb_inst = getattr(comp_obj, ext_algeb_name)
                                     if hasattr(ext_algeb_inst, 'e'):
                                         print(f"    {comp_name}.{ext_algeb_name}.e exists, length {len(ext_algeb_inst.e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    case_file_to_load = "YOUR_CASE_FILE.raw"
    try:
        if case_file_to_load == "YOUR_CASE_FILE.raw":
             print("Using a default IEEE 14 bus case from ANDES library for demonstration as 'YOUR_CASE_FILE.raw' was not changed.")
             print("Please replace 'YOUR_CASE_FILE.raw' with your actual file path to analyze your specific system.")
             case_file_to_load = andes.get_case("ieee14/ieee14.raw")
    except Exception as e:
        print(f"Could not load default case using andes.get_case: {e}.")
        print("Please ensure 'YOUR_CASE_FILE.raw' is replaced with a valid file path if the default case is not found or andes.get_case is unavailable.")
        import sys
        if case_file_to_load == "YOUR_CASE_FILE.raw":
            sys.exit(1)

    run_power_flow_and_print_results(case_file_to_load)

    print("\nNote: If some data is missing or values are 'nan', investigate the specific variable/parameter/service names for that model in the ANDES source or documentation.")
    print("  - For generators/loads, check algebraic variables (Algeb) like 'p', 'q', 'Pe', 'Qe' or services like 'Ppf', 'Qpf'. Their values are usually in the '.v' attribute.")
    print("  - For lines, power injections at terminals are typically in the '.e' attribute of ExtAlgeb variables like 'a1', 'v1' (for bus1) and 'a2', 'v2' (for bus2).")

INFO:andes:Working directory: "/content"
DEBUG:andes:Found files: ['/usr/local/lib/python3.11/dist-packages/andes/cases/ieee14/ieee14.raw']


Using a default IEEE 14 bus case from ANDES library for demonstration as 'YOUR_CASE_FILE.raw' was not changed.
Please replace 'YOUR_CASE_FILE.raw' with your actual file path to analyze your specific system.
Loading case file: /usr/local/lib/python3.11/dist-packages/andes/cases/ieee14/ieee14.raw


INFO:andes.system:> Reloaded generated Python code of module "pycode".
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
DEBUG:andes.io.psse:PSSE raw version 32 detected
DEBUG:andes.io:Input format guessed as psse.
INFO:andes.io:Parsing input file "/usr/local/lib/python3.11/dist-packages/andes/cases/ieee14/ieee14.raw"...
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
INFO:andes.io.psse:  IEEE 14 BUS TEST CASE
INFO:andes.io.psse:  03/06/14 CONTO           100.0  1962 W
INFO:andes.io:Input file parsed in 0.0276 seconds.
DEBUG:andes.system:Setting internal address for Bus
DEBUG:andes.system:Setting internal address for PQ
DEBUG:andes.system:Setting internal address for PV
DEBUG:andes.system:Setting internal address for Slack
DEBUG:andes.system:Setting internal address for Shunt
DEBUG:andes.system:Setting internal address for Line
DEBUG:andes.system:Setting internal address for Area
DEBUG

Case file loaded successfully.

Running Power Flow...
Power flow calculation completed successfully.

--- Generator Powers ---

PV Generators:
  Generator: 2 (ID: 2), Bus: 2, P: 0.4000 MW (setpoint), Q: 0.3044 MVAr (solved)
  Generator: 3 (ID: 3), Bus: 3, P: 0.4000 MW (setpoint), Q: 0.1260 MVAr (solved)
  Generator: 4 (ID: 4), Bus: 6, P: 0.3000 MW (setpoint), Q: 0.2099 MVAr (solved)
  Generator: 5 (ID: 5), Bus: 8, P: 0.3500 MW (setpoint), Q: 0.0740 MVAr (solved)

Slack Generators:
  Generator: 1 (ID: 1), Bus: 1, P: 0.8143 MW (solved), Q: -0.2162 MVAr (solved)

  No GENROU generators found in the case.

  No GENCLS generators found in the case.

StaticGen Generators (distinct type):
  Generator: 2.0 (ID: 2.0), Bus: 2.0, P: 0.4000 MW, Q: 0.1500 MVAr
  Generator: 3.0 (ID: 3.0), Bus: 3.0, P: 0.4000 MW, Q: 0.1500 MVAr
  Generator: 4.0 (ID: 4.0), Bus: 6.0, P: 0.3000 MW, Q: 0.1000 MVAr
  Generator: 5.0 (ID: 5.0), Bus: 8.0, P: 0.3500 MW, Q: 0.1000 MVAr
  Generator: 1.0 (ID: 1.0), Bus: 1.0, P: 

In [62]:
import andes
import pandas as pd
import numpy as np
import os # Import os for file existence check

# Attempt to import GroupBase for the isinstance check in the exception handler
try:
    from andes.models.group import GroupBase
except ImportError:
    print("Warning: Could not import GroupBase from andes.models.group. Type checking for groups in debug output may be affected.")
    GroupBase = object # Fallback to object to avoid NameError, isinstance will just be less specific

def normalize_idx_to_str(idx_val):
    """
    Normalizes an index value to a string. If it's a float representing a whole number,
    it's converted to the string of the integer. Otherwise, it's the standard string representation.
    """
    if isinstance(idx_val, float) and idx_val.is_integer():
        return str(int(idx_val))
    return str(idx_val)

def run_power_flow_and_print_results(case_file_path):
    """
    Loads a power system case, runs power flow, and prints results
    for generator powers, load powers, and line flows.
    Returns a dictionary of results if successful, else None.
    """
    ss = None
    line_model_name = 'Line'
    results = {
        "generators": [],
        "loads": [],
        "lines": [],
        "buses": [],
        "node_name_map": {} # For consistent naming in the flow matrix
    }
    try:
        print(f"Loading case file: {case_file_path}")
        ss = andes.load(case_file_path, setup=True, no_output=True)
        print("Case file loaded successfully.")

        print("\nRunning Power Flow...")
        pf_success = ss.PFlow.run()

        if not pf_success:
            print(f"Power flow calculation did not converge or failed. Exit code: {getattr(ss.PFlow, 'exit_code', 'N/A')}")
            return None
        print("Power flow calculation completed successfully.")

        def get_values_as_series(data_provider_object, value_attribute_name, index_provider_model):
            if data_provider_object is None:
                return pd.Series(dtype=float)
            if not (index_provider_model and hasattr(index_provider_model, 'idx') and hasattr(index_provider_model.idx, 'v')):
                return pd.Series(dtype=float)

            if hasattr(data_provider_object, value_attribute_name):
                value_array = getattr(data_provider_object, value_attribute_name)
                idx_array_raw = index_provider_model.idx.v
                idx_array = [str(item) for item in idx_array_raw] # Index for series created using original str(idx)

                if isinstance(value_array, (np.ndarray, list)) and len(value_array) == len(idx_array):
                    try:
                        if not idx_array: return pd.Series(dtype=float)
                        return pd.Series(data=value_array, index=idx_array)
                    except Exception:
                        try:
                            return pd.Series(data=value_array, index=list(idx_array))
                        except Exception:
                            pass
            return pd.Series(dtype=float)

        node_name_map = results["node_name_map"]
        processed_gen_original_ids = set() # This will store normalized IDs

        print("\n--- Generator Powers ---")

        gen_model_processing_order = [
            {'model_name': 'PV', 'p_var': 'p0', 'q_var': 'q', 'p_is_param': True},
            {'model_name': 'Slack', 'p_var': 'p', 'q_var': 'q', 'p_is_param': False},
            {'model_name': 'GENROU', 'p_var': 'Pe', 'q_var': 'Qe', 'p_is_param': False},
            {'model_name': 'GENCLS', 'p_var': 'Pe', 'q_var': 'Qe', 'p_is_param': False}
        ]

        for gen_type_config in gen_model_processing_order:
            model_name = gen_type_config['model_name']
            if hasattr(ss, model_name) and getattr(ss, model_name).n > 0:
                print(f"\n{model_name} Generators:")
                model_obj = getattr(ss, model_name)
                df_full = model_obj.as_df()

                p_data_obj = getattr(model_obj, gen_type_config['p_var'], None)
                q_data_obj = getattr(model_obj, gen_type_config['q_var'], None)

                p_series = get_values_as_series(p_data_obj, 'v', model_obj)
                q_series = get_values_as_series(q_data_obj, 'v', model_obj)

                for _, row in df_full.iterrows():
                    normalized_id_str = normalize_idx_to_str(row['idx'])

                    if normalized_id_str in processed_gen_original_ids:
                        continue
                    processed_gen_original_ids.add(normalized_id_str)

                    display_name = row.get('name', normalized_id_str)
                    matrix_node_name = f"Gen_{normalized_id_str}"
                    node_name_map[('Generator', normalized_id_str)] = matrix_node_name

                    original_idx_str_for_series_lookup = str(row['idx'])
                    p_val = p_series.get(original_idx_str_for_series_lookup, float('nan'))
                    q_val = q_series.get(original_idx_str_for_series_lookup, float('nan'))

                    gen_data = {'type': model_name, 'matrix_name': matrix_node_name,
                                'original_name': display_name, 'id': normalized_id_str,
                                'bus': str(row.get('bus')), 'P': p_val, 'Q': q_val}
                    results["generators"].append(gen_data)
                    p_desc = "(setpoint)" if gen_type_config['p_is_param'] else "(solved)"
                    print(f"  Generator: {display_name} (ID: {normalized_id_str}), Bus: {row.get('bus', 'N/A')}, "
                          f"P: {p_val:.4f} MW {p_desc}, Q: {q_val:.4f} MVAr (solved)")
            else:
                print(f"\n  No {model_name} generators found in the case.")

        if hasattr(ss, 'StaticGen') and ss.StaticGen.n > 0:
            sg_df_full = ss.StaticGen.as_df()
            newly_added_static_gens = []
            if 'idx' in sg_df_full.columns:
                for _, row in sg_df_full.iterrows():
                    normalized_id_str = normalize_idx_to_str(row['idx'])

                    if normalized_id_str not in processed_gen_original_ids:
                        processed_gen_original_ids.add(normalized_id_str)

                        display_name = row.get('name', normalized_id_str)
                        matrix_node_name = f"Gen_{normalized_id_str}"
                        node_name_map[('StaticGen', normalized_id_str)] = matrix_node_name

                        p_val = row.get('p0', float('nan'))
                        q_val = row.get('q0', float('nan'))
                        gen_data = {'type': 'StaticGen_unique',
                                    'matrix_name': matrix_node_name,
                                    'original_name': display_name, 'id': normalized_id_str,
                                    'bus': str(row.get('bus')), 'P': p_val, 'Q': q_val}
                        results["generators"].append(gen_data)
                        newly_added_static_gens.append(gen_data)

            if newly_added_static_gens:
                print("\nStaticGen Generators (unique instances not previously listed):")
                for gen_data in newly_added_static_gens:
                    print(f"  Generator: {gen_data['original_name']} (ID: {gen_data['id']}), Bus: {gen_data['bus']}, "
                          f"P: {gen_data['P']:.4f} MW (from p0), Q: {gen_data['Q']:.4f} MVAr (from q0)")

        print("\n--- Load Powers (PQ model) ---")
        if hasattr(ss, 'PQ') and ss.PQ.n > 0:
            pq_df_full = ss.PQ.as_df()
            p_series = get_values_as_series(ss.PQ.Ppf, 'v', ss.PQ)
            q_series = get_values_as_series(ss.PQ.Qpf, 'v', ss.PQ)

            for _, row in pq_df_full.iterrows():
                dev_id_str = str(row['idx'])
                original_name = row.get('name', dev_id_str)
                p_val = p_series.get(dev_id_str, float('nan'))
                q_val = q_series.get(dev_id_str, float('nan'))
                load_data = {'original_name': original_name, 'id': dev_id_str,
                             'bus': str(row.get('bus')), 'P': p_val, 'Q': q_val}
                results["loads"].append(load_data)
                print(f"  Load: {original_name} (ID: {dev_id_str}), Bus: {row.get('bus', 'N/A')}, "
                      f"P: {p_val:.4f} MW, Q: {q_val:.4f} MVAr")
        else:
            print("  No PQ loads found in the case.")

        if hasattr(ss, 'Bus') and ss.Bus.n > 0:
            bus_df_full = ss.Bus.as_df()
            vm_series = get_values_as_series(ss.Bus.v, 'v', ss.Bus)
            va_series = get_values_as_series(ss.Bus.a, 'v', ss.Bus)

            for _, row in bus_df_full.iterrows():
                dev_id_str = str(row['idx'])
                original_name = row.get('name', dev_id_str)
                try:
                    numeric_part_str = "".join(filter(str.isdigit, original_name))
                    sort_key = int(numeric_part_str) if numeric_part_str else original_name
                except ValueError:
                    sort_key = original_name

                matrix_node_name = f"Bus_{original_name}"
                node_name_map[('Bus', dev_id_str)] = matrix_node_name

                vm_val = vm_series.get(dev_id_str, float('nan'))
                va_val = va_series.get(dev_id_str, float('nan'))
                results["buses"].append({'matrix_name': matrix_node_name,
                                         'original_name': original_name, 'id': dev_id_str,
                                         'Vm': vm_val, 'Va': va_val, '_sort_key': sort_key})

        line_model_name = 'Line'
        if not hasattr(ss, line_model_name) or getattr(ss, line_model_name).n == 0:
            line_model_name = 'ACLine'

        print(f"\n--- Line Power Flows ({line_model_name} model) ---")
        if hasattr(ss, line_model_name) and getattr(ss, line_model_name).n > 0:
            line_model_obj = getattr(ss, line_model_name)
            line_df_full = line_model_obj.as_df()

            p_inj_bus1_series = get_values_as_series(line_model_obj.a1, 'e', line_model_obj)
            q_inj_bus1_series = get_values_as_series(line_model_obj.v1, 'e', line_model_obj)
            p_inj_bus2_series = get_values_as_series(line_model_obj.a2, 'e', line_model_obj)
            q_inj_bus2_series = get_values_as_series(line_model_obj.v2, 'e', line_model_obj)

            for _, row in line_df_full.iterrows():
                dev_id_str = str(row['idx'])
                original_name = row.get('name', dev_id_str)
                bus1 = str(row.get('bus1','N/A'))
                bus2 = str(row.get('bus2','N/A'))

                p_inj_bus1 = p_inj_bus1_series.get(dev_id_str, float('nan'))
                q_inj_bus1 = q_inj_bus1_series.get(dev_id_str, float('nan'))
                p_inj_bus2 = p_inj_bus2_series.get(dev_id_str, float('nan'))
                q_inj_bus2 = q_inj_bus2_series.get(dev_id_str, float('nan'))

                line_data = {'original_name': original_name, 'id': dev_id_str,
                             'bus1': bus1, 'bus2': bus2,
                             'P_inj_bus1': p_inj_bus1, 'Q_inj_bus1': q_inj_bus1,
                             'P_inj_bus2': p_inj_bus2, 'Q_inj_bus2': q_inj_bus2}
                results["lines"].append(line_data)

                print(f"  Line: {original_name} (ID: {dev_id_str}) (Bus1: {bus1} to Bus2: {bus2})")
                print(f"    Injection at Bus {bus1} from Line: P = {p_inj_bus1:.4f} MW, Q = {q_inj_bus1:.4f} MVAr")
                print(f"    Injection at Bus {bus2} from Line: P = {p_inj_bus2:.4f} MW, Q = {q_inj_bus2:.4f} MVAr")
        else:
            print(f"  No {line_model_name} model found or no instances in the case.")

        return results

    except FileNotFoundError:
        print(f"Error: Case file not found at '{case_file_path}'.")
    except ImportError as ie:
        print(f"Error: ANDES library or a dependency not found: {ie}.")
    except AttributeError as ae:
        print(f"An AttributeError occurred: {ae}.")
        import traceback
        traceback.print_exc()
        if ss and hasattr(ss, 'PFlow') and hasattr(ss.PFlow, 'exit_code') and ss.PFlow.exit_code != 0 :
             print(f"Power flow convergence issue might be the cause. Exit code: {ss.PFlow.exit_code}")
        elif ss and GroupBase:
            print("Debug: Available components in the system object (ss):")
            for comp_name in dir(ss):
                if not comp_name.startswith('_'):
                    comp_obj = getattr(ss, comp_name, None)
                    if isinstance(comp_obj, GroupBase) and hasattr(comp_obj, 'n') and comp_obj.n > 0:
                        print(f"  - {comp_name} (Instances: {comp_obj.n}, Type: {type(comp_obj)})")
                        for var_container_name in ['algebs', 'params', 'services', 'states', 'vars_ext', 'vars_int']:
                            if hasattr(comp_obj, var_container_name):
                                container = getattr(comp_obj, var_container_name)
                                if container and isinstance(container, dict) and container:
                                    print(f"     {var_container_name}: {list(container.keys())}")
                        if 'line_model_name' in locals() and comp_name == line_model_name:
                            for ext_algeb_name in ['a1', 'v1', 'a2', 'v2']:
                                if hasattr(comp_obj, ext_algeb_name):
                                    ext_algeb_inst = getattr(comp_obj, ext_algeb_name)
                                    if hasattr(ext_algeb_inst, 'e'):
                                        val_to_print = ext_algeb_inst.e if ext_algeb_inst.e is not None else []
                                        print(f"     {comp_name}.{ext_algeb_name}.e exists, length {len(val_to_print)}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
    return None


def create_flow_matrix(pf_results):
    if pf_results is None:
        print("Power flow results are not available. Cannot create flow matrix.")
        return None

    node_name_map = pf_results.get("node_name_map", {})

    gen_matrix_names_set = set()
    for g in pf_results["generators"]:
        gen_matrix_names_set.add(g['matrix_name'])
    gen_nodes_sorted = sorted(list(gen_matrix_names_set))

    buses_sorted_for_matrix = sorted(pf_results["buses"], key=lambda x: x.get('_sort_key', x['original_name']))
    bus_nodes_sorted = [b['matrix_name'] for b in buses_sorted_for_matrix]

    internal_nodes = gen_nodes_sorted + bus_nodes_sorted
    compartment_order = ["Input"] + internal_nodes + ["Total_Load", "Total_Dissipation"]

    flow_matrix = pd.DataFrame(0.0, index=compartment_order, columns=compartment_order)

    for gen_data in pf_results["generators"]:
        gen_matrix_node = gen_data['matrix_name']
        p_gen = gen_data.get('P', 0.0)
        if pd.notna(p_gen) and p_gen > 1e-6:
            if "Input" in flow_matrix.index and gen_matrix_node in flow_matrix.columns:
                flow_matrix.loc["Input", gen_matrix_node] = p_gen

    for gen_data in pf_results["generators"]:
        gen_matrix_node = gen_data['matrix_name']
        bus_id_of_gen = gen_data['bus']

        bus_matrix_node = node_name_map.get(('Bus', bus_id_of_gen))
        if not bus_matrix_node:
            for b_info in pf_results["buses"]:
                if str(b_info["id"]) == bus_id_of_gen:
                    bus_matrix_node = b_info["matrix_name"]
                    break

        p_gen = gen_data.get('P', 0.0)
        if pd.notna(p_gen) and p_gen > 1e-6:
            if gen_matrix_node in flow_matrix.index and bus_matrix_node and bus_matrix_node in flow_matrix.columns:
                flow_matrix.loc[gen_matrix_node, bus_matrix_node] = p_gen

    for line in pf_results["lines"]:
        bus1_id_original = line['bus1']
        bus2_id_original = line['bus2']

        bus1_matrix_node = node_name_map.get(('Bus', bus1_id_original))
        bus2_matrix_node = node_name_map.get(('Bus', bus2_id_original))
        if not bus1_matrix_node:
            for b_info in pf_results["buses"]:
                if str(b_info["id"]) == bus1_id_original: bus1_matrix_node = b_info["matrix_name"]; break
        if not bus2_matrix_node:
            for b_info in pf_results["buses"]:
                if str(b_info["id"]) == bus2_id_original: bus2_matrix_node = b_info["matrix_name"]; break

        if not bus1_matrix_node or not bus2_matrix_node: continue

        p_inj_bus1 = line.get('P_inj_bus1', 0.0)

        if pd.notna(p_inj_bus1) and p_inj_bus1 < -1e-6:
            flow_val = abs(p_inj_bus1)
            if bus1_matrix_node in flow_matrix.index and bus2_matrix_node in flow_matrix.columns:
                flow_matrix.loc[bus1_matrix_node, bus2_matrix_node] += flow_val
        elif pd.notna(p_inj_bus1) and p_inj_bus1 > 1e-6:
            flow_val = p_inj_bus1
            if bus2_matrix_node in flow_matrix.index and bus1_matrix_node in flow_matrix.columns:
                 flow_matrix.loc[bus2_matrix_node, bus1_matrix_node] += flow_val

    if "Total_Load" in flow_matrix.columns:
        for load in pf_results["loads"]:
            bus_id_of_load = load['bus']
            bus_matrix_node = node_name_map.get(('Bus', bus_id_of_load))
            if not bus_matrix_node:
                for b_info in pf_results["buses"]:
                    if str(b_info["id"]) == bus_id_of_load: bus_matrix_node = b_info["matrix_name"]; break

            load_p = load.get('P', 0.0)
            if bus_matrix_node and bus_matrix_node in flow_matrix.index and pd.notna(load_p) and load_p > 1e-6:
                flow_matrix.loc[bus_matrix_node, "Total_Load"] += load_p

    bus_total_dissipation = {bus_mn: 0.0 for bus_mn in bus_nodes_sorted}
    if "Total_Dissipation" in flow_matrix.columns:
        for line in pf_results["lines"]:
            p_inj_bus1_val = line.get('P_inj_bus1', 0.0) if pd.notna(line.get('P_inj_bus1')) else 0.0
            p_inj_bus2_val = line.get('P_inj_bus2', 0.0) if pd.notna(line.get('P_inj_bus2')) else 0.0

            current_line_loss = -(p_inj_bus1_val + p_inj_bus2_val)

            if current_line_loss > 1e-6:
                bus1_id_original = line['bus1']
                bus2_id_original = line['bus2']
                bus1_matrix_node = node_name_map.get(('Bus', bus1_id_original))
                bus2_matrix_node = node_name_map.get(('Bus', bus2_id_original))
                if not bus1_matrix_node:
                    for b_info in pf_results["buses"]:
                        if str(b_info["id"]) == bus1_id_original: bus1_matrix_node = b_info["matrix_name"]; break
                if not bus2_matrix_node:
                    for b_info in pf_results["buses"]:
                        if str(b_info["id"]) == bus2_id_original: bus2_matrix_node = b_info["matrix_name"]; break

                if bus1_matrix_node and bus1_matrix_node in bus_total_dissipation:
                    bus_total_dissipation[bus1_matrix_node] += current_line_loss / 2.0
                if bus2_matrix_node and bus2_matrix_node in bus_total_dissipation:
                    bus_total_dissipation[bus2_matrix_node] += current_line_loss / 2.0

        for bus_node_name, diss_val in bus_total_dissipation.items():
            if bus_node_name in flow_matrix.index and diss_val > 0:
                flow_matrix.loc[bus_node_name, "Total_Dissipation"] += diss_val

    for node_label in flow_matrix.index:
        if node_label in flow_matrix.columns:
            flow_matrix.loc[node_label, node_label] = 0.0

    print("\n--- Flow Matrix (Active Power P) ---")
    pd.set_option('display.width', 200)
    pd.set_option('display.max_columns', len(flow_matrix.columns) + 2)
    print(flow_matrix.round(4))
    return flow_matrix


if __name__ == '__main__':
    case_file_to_load = None
    while True:
        user_input_path = input("Please enter the full path to your .raw case file: ").strip()
        if not user_input_path:
            print("No path entered. Please try again or type 'exit' to quit.")
            continue
        if user_input_path.lower() == 'exit':
            print("Exiting script.")
            import sys
            sys.exit(0)

        # Basic check for .raw extension, can be made more robust
        if not user_input_path.lower().endswith(".raw"):
            print("The file path does not seem to end with '.raw'. Please ensure it's a PSS/E raw file.")
            # Optionally, ask for confirmation to proceed if user is sure
            confirm = input("Do you want to proceed with this path anyway? (yes/no): ").strip().lower()
            if confirm != 'yes':
                continue

        # Check if file exists
        if os.path.exists(user_input_path) and os.path.isfile(user_input_path):
            case_file_to_load = user_input_path
            break
        else:
            print(f"Error: File not found at '{user_input_path}'. Please check the path and try again.")
            # It's good practice to also check if andes is available early
            try:
                import andes
            except ImportError:
                print("Critical Error: ANDES library is not installed. This script cannot run.")
                print("Please install ANDES (e.g., pip install andes) and try again.")
                import sys
                sys.exit(1)


    if case_file_to_load:
        power_flow_results_data = run_power_flow_and_print_results(case_file_to_load)

        if power_flow_results_data:
            print("\nConstructing Flow Matrix...")
            flow_matrix = create_flow_matrix(power_flow_results_data)
            if flow_matrix is not None:
                 print("\nFlow matrix construction process complete.")
    else:
        # This case should ideally not be reached if the loop is exited with a valid file or 'exit'
        print("No case file was loaded. Exiting.")

    print("\n--- End of Script ---")

Please enter the full path to your .raw case file: /content/wscc9.raw


INFO:andes:Working directory: "/content"
DEBUG:andes:Found files: ['/content/wscc9.raw']


Loading case file: /content/wscc9.raw


INFO:andes.system:> Reloaded generated Python code of module "pycode".
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
DEBUG:andes.io.psse:PSSE raw version 33 detected
DEBUG:andes.io:Input format guessed as psse.
INFO:andes.io:Parsing input file "/content/wscc9.raw"...
DEBUG:andes.io:Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
INFO:andes.io:Input file parsed in 0.0138 seconds.
DEBUG:andes.system:Setting internal address for Bus
DEBUG:andes.system:Setting internal address for PQ
DEBUG:andes.system:Setting internal address for PV
DEBUG:andes.system:Setting internal address for Slack
DEBUG:andes.system:Setting internal address for Line
DEBUG:andes.system:Setting internal address for Area
DEBUG:andes.core.connman:No need to update connectivity.
INFO:andes.system:System internal structure set up in 0.0592 seconds.
DEBUG:andes.system:Entering connectivity check.
INFO:andes.system:-> System connectivi

Case file loaded successfully.

Running Power Flow...
Power flow calculation completed successfully.

--- Generator Powers ---

PV Generators:
  Generator: 2 (ID: 2), Bus: 2, P: 1.6300 MW (setpoint), Q: 0.0490 MVAr (solved)
  Generator: 3 (ID: 3), Bus: 3, P: 0.8500 MW (setpoint), Q: -0.1145 MVAr (solved)

Slack Generators:
  Generator: 1 (ID: 1), Bus: 1, P: 0.7163 MW (solved), Q: 0.2791 MVAr (solved)

  No GENROU generators found in the case.

  No GENCLS generators found in the case.

--- Load Powers (PQ model) ---
  Load: PQ_1 (ID: PQ_1), Bus: 5, P: 1.2500 MW, Q: 0.5000 MVAr
  Load: PQ_2 (ID: PQ_2), Bus: 6, P: 0.9000 MW, Q: 0.3000 MVAr
  Load: PQ_3 (ID: PQ_3), Bus: 8, P: 1.0000 MW, Q: 0.3500 MVAr

--- Line Power Flows (Line model) ---
  Line: Line_1 (ID: Line_1) (Bus1: 5 to Bus2: 4)
    Injection at Bus 5 from Line: P = -0.4301 MW, Q = -0.3959 MVAr
    Injection at Bus 4 from Line: P = 0.4329 MW, Q = 0.2345 MVAr
  Line: Line_2 (ID: Line_2) (Bus1: 6 to Bus2: 4)
    Injection at Bus 6 